In [1]:
## 위클리 상품 이미지 다운받기
# 실행(▶ Run)하고 위클리아이디를 입력하시면 해당하는 위클리의 검수된 제안서의 제품사진을 다운받으실 수있습니다.
# 다운받은 파일은 해당하는 이미지 폴더 내의 위클리아이디로 폴더가 만들어지고 해당하는 폴더 하위에 생성됩니다.
# 파일명은 "위클리아이디_다운받은날짜.zip" 입니다.
# 해당하는 .zip파일을 클릭한 후 Download를 클릭하시면 다운로드됩니다.

In [ ]:
import os
import shutil
import datetime
import sys

sys.path.append('/home/libs')
config = __import__('config')

# 위클리 데이터 기본키 입력 받기
weekly_id = input("위클리 기본키: ")
current_date = datetime.datetime.now().strftime('%Y%m%d')

In [ ]:
# 쿼리문 실행
cur = config.conn.cursor()
cur.execute("""
    SELECT sp.id as suggestion_product_id, hyperlink, image, ss.id
    FROM "Suggestion_product" sp
    LEFT JOIN "Suggestion_suggestion" ss ON sp.suggestion_id = ss.id
    WHERE ss.id IN (
              SELECT ss.id FROM "Suggestion_suggestion" ss
    LEFT JOIN "Request_request" rr ON ss.id = rr.suggestion_id
    INNER JOIN styling_weeklystylinginfo sw ON rr.id = sw.request_id
    WHERE sw.weekly_info_id =%s
    AND ss.is_confirmed = true)
AND ss.thumbnail = ''
""", (weekly_id,))

queryResult = cur.fetchall()

In [ ]:
# 제안서 아이디를 키값으로 오브젝트 생성
suggestion_obj = {}
for row in queryResult:
    suggestion_id = row[3]
    if suggestion_id not in suggestion_obj:
        suggestion_obj[suggestion_id] = []
    suggestion_obj[suggestion_id].append(row)

In [ ]:
# 제안서 아이디로 만든 오브젝트를 통해서 S3에서 이미지를 받아옴 
download_images_count = 0;

for suggestion_id, obj in suggestion_obj.items():
    # product_infos.txt는 제품 착샷을 빠르게 찾기위해 작성되는 파일
    filepath = f"./이미지/{weekly_id}/{weekly_id}_{current_date}/{suggestion_id}/product_infos.txt"
    
    try:
        # suggestion_obj로 폴더를 생성
        folder_path = f"./이미지/{weekly_id}/{weekly_id}_{current_date}/{suggestion_id}"
        os.makedirs(folder_path, exist_ok=False)
    except:
        continue

    try:
        download_images_count += 1;
        with open(filepath, "w", encoding='UTF-8') as f:
            
            # S3에 접근하여 제안서 아이디로 파일을 읽어 로컬에 저장(있으면 계속, 없을때 저장)
            os.system(
                f'aws s3 cp s3://stylerecipe-korea/media/suggestion/{suggestion_id}/ ./이미지/{weekly_id}/{weekly_id}_{current_date}/{suggestion_id}/ --recursive')
            for data in obj:
                toimage = f'./이미지/{weekly_id}/{weekly_id}_{current_date}/{suggestion_id}/{data[0]}'
                exp = data[2].split('.')[-1]
              
                # 제안서에서 생성된 이미지가 아닌 통합상품에서 들고오는 이미지는 이미지 경로가 product일 수 있음.
                if data[2].split('/')[0] == 'product':
                    os.system(
                        f'aws s3 cp s3://stylerecipe-korea/media/{data[2]} {toimage}.{exp} ')
                elif data[2].split('/')[1] != str(suggestion_id):
                    os.system(
                        f'aws s3 cp s3://stylerecipe-korea/media/{data[2]} {toimage}.{exp} ')
                else:
                    image = data[2].split('/')[-1]
                    fromimage = f'./이미지/{weekly_id}/{weekly_id}_{current_date}/{suggestion_id}/{image}'
                    try:
                        os.system(f'mv {fromimage} {toimage}.{exp}')
                    except:
                        pass
                
                f.write(
                    f'{data[0]} : {data[1]}\n')
                
    except Exception as e:
        print(e)
        pass
    
print("download_images_count:", download_images_count)

In [6]:
# 최종본 zip파일로 저장
shutil.make_archive(f'./이미지/{weekly_id}/{weekly_id}_{current_date}','zip',f'./이미지/{weekly_id}/{weekly_id}_{current_date}')

'/home/jovyan/위클리스타일링/이미지/43/43_20230424.zip'

In [7]:
os.system(f'rm -r ./이미지/{weekly_id}/{weekly_id}_{current_date}')

0